In [1]:
import numpy as np
import os
import pandas as pd
import re

In [2]:
episodes = []
for (dirpath, dirnames, filenames) in os.walk('C:\\Users\\Nirvan S P Theethira\\Desktop\\Pictionary\\data\\transcripts_friends\\season_all\\'):
    for file in filenames:
        if '.csv' in file:
            episodes.append(dirpath+file)

In [3]:
episodes[9]

'C:\\Users\\Nirvan S P Theethira\\Desktop\\Pictionary\\data\\transcripts_friends\\season_all\\101.csv'

In [4]:
with open(episodes[9],encoding='utf8') as f:
    lines = f.readlines()

In [8]:
lines

['The One Where Monica Gets a New Roommate (The Pilot-The Uncut Version)\n',
 'Written by: Marta Kauffman & David Crane\n',
 'Transcribed by: guineapig\n',
 'Additional transcribing by: Eric Aasen\n',
 '(Note: The previously unseen parts of this episode are shown in blue text.)\n',
 '\n',
 '[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\n',
 '\n',
 "Monica: There's nothing to tell! He's just some guy I work with!\n",
 '\n',
 "Joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!\n",
 '\n',
 'Chandler: All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?\n',
 '\n',
 'Phoebe: Wait, does he eat chalk?\n',
 '\n',
 '(They all stare, bemused.)\n',
 '\n',
 "Phoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!\n",
 '\n',
 "Monica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.\n",
 '\n',
 'Chandler: Sounds like a date to m

In [9]:
line = 'Monica: (really excited) Mmh... this cake is amazing!'.lower()
line

'monica: (really excited) mmh... this cake is amazing!'

In [13]:
re.search('^[a-z]+:','monica: (really excited) mmh: ... this cake is amazing!')

<re.Match object; span=(0, 7), match='monica:'>

In [14]:
re.findall('^[a-z]+:','monica: (really excited) mmh ... this cake is amazing!')

['monica:']

In [15]:
re.sub(':','','monica:')

'monica'

In [17]:
scene = ''
preChar = ''
dialogue = ''
dataF = {'scene':[],'person':[], 'dialogue':[]}
for line in lines:
    line = re.sub('\n','',line).lower()
    if re.search(r"^\[[\w\W]+\]$", re.sub('\n','',line)):
        scene = line
    if re.search('^[a-z]+:',line) and len(scene)>0:
        charachter = re.findall('^[a-z]+:',line)[0]
        charachter = re.sub(':','',charachter)
        dialogue = re.sub('^[a-z]+:','',line)
        dataF['scene'].append(scene)
        dataF['person'].append(charachter)
        dataF['dialogue'].append(dialogue)

In [20]:
pd.DataFrame(dataF)

,scene,person,dialogue
0,"[scene: central perk, chandler, joey, phoebe, ...",monica,there's nothing to tell! he's just some guy i...
1,"[scene: central perk, chandler, joey, phoebe, ...",joey,"c'mon, you're going out with the guy! there's..."
2,"[scene: central perk, chandler, joey, phoebe, ...",chandler,"all right joey, be nice. so does he have a h..."
3,"[scene: central perk, chandler, joey, phoebe, ...",phoebe,"wait, does he eat chalk?"
4,"[scene: central perk, chandler, joey, phoebe, ...",phoebe,"just, 'cause, i don't want her to go through ..."
5,"[scene: central perk, chandler, joey, phoebe, ...",monica,"okay, everybody relax. this is not even a dat..."
6,"[scene: central perk, chandler, joey, phoebe, ...",chandler,sounds like a date to me.
7,[time lapse],chandler,"alright, so i'm back in high school, i'm stan..."
8,[time lapse],all,"oh, yeah. had that dream."
9,[time lapse],chandler,"then i look down, and i realize there's a pho..."
